In [1]:
import pandas as pd


In [8]:
computer_name1 = 'gilbe'
computer_name2 = 'Gilberto-BE'

ROOT_PATH = f'c:/Users/{computer_name1}/Documents/TokyoData'


'/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv'
'/train_files/trades.csv'

train_df = pd.read_csv(f'{ROOT_PATH}/train_files/stock_prices.csv')
train_df['Date'] = pd.to_datetime(train_df['Date']) 
train_df.set_index('Date', inplace=True)
# train_df = date_features(train_df)
print(train_df.head(2))

train_options = pd.read_csv(f'{ROOT_PATH}/train_files/options.csv', low_memory=False)
train_financials = pd.read_csv(f'{ROOT_PATH}/train_files/financials.csv', low_memory=False)
train_trades = pd.read_csv(f'{ROOT_PATH}/train_files/trades.csv', low_memory=False)


print(train_df.head(2))
print(train_df.info())
print('.' * 20)
print()


print(train_options.head(2))
print(train_options.info())
print('.' * 20)
print()


print(train_financials.head(2))
print(train_financials.info())
print('.' * 20)
print()


print(train_trades.head(2))
print(train_trades.info())
print('.' * 20)
print()


                    RowId  SecuritiesCode    Open    High     Low   Close  \
Date                                                                        
2017-01-04  20170104_1301            1301  2734.0  2755.0  2730.0  2742.0   
2017-01-04  20170104_1332            1332   568.0   576.0   563.0   571.0   

             Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag  \
Date                                                                       
2017-01-04    31400               1.0               NaN            False   
2017-01-04  2798500               1.0               NaN            False   

              Target  
Date                  
2017-01-04  0.000730  
2017-01-04  0.012324  
                    RowId  SecuritiesCode    Open    High     Low   Close  \
Date                                                                        
2017-01-04  20170104_1301            1301  2734.0  2755.0  2730.0  2742.0   
2017-01-04  20170104_1332            1332   568.0   576.0   563